In [ ]:
import pandas as pd
import re
from collections import defaultdict
from fuzzywuzzy import fuzz
import numpy as np

# Load the data
file_path = r"C:\Users\spt-admin\Desktop\NEWD\schedule_records.csv"
df = pd.read_csv(file_path)

print(f"Loaded {len(df)} rows")
print(f"Columns: {df.columns.tolist()}")

# Step 1: Clean SSNIT numbers
def clean_ssnit(ssnit):
    if pd.isna(ssnit):
        return ''
    # Convert to string, strip whitespace, remove special characters
    ssnit = str(ssnit).strip()
    ssnit = re.sub(r'[^\w]', '', ssnit)  # Remove non-alphanumeric
    return ssnit.upper()

df['CLEANED_SSNIT'] = df['SSNIT_NUMBER'].apply(clean_ssnit)

# Check if SSNIT is invalid (empty, 0, unknown, nan, etc.)
def is_invalid_ssnit(ssnit):
    if pd.isna(ssnit) or ssnit == '':
        return True
    ssnit_str = str(ssnit).strip().upper()
    
    # Exact matches for invalid values
    invalid_values = ['', '0', 'UNKNOWN', 'NAN', 'NA', 'N/A']
    if ssnit_str in invalid_values:
        return True
    
    # Check if it contains certain keywords (TRUST, UNKNOWN, NO_SSF_NUMBER)
    invalid_keywords = ['TRUST', 'UNKNOWN', 'NO_SSF_NUMBER']
    for keyword in invalid_keywords:
        if keyword in ssnit_str:
            return True
    
    return False

df['INVALID_SSNIT'] = df['CLEANED_SSNIT'].apply(is_invalid_ssnit)

print(f"\nUnique original SSNIT: {df['SSNIT_NUMBER'].nunique()}")
print(f"Unique cleaned SSNIT: {df['CLEANED_SSNIT'].nunique()}")
print(f"Records with invalid SSNIT: {df['INVALID_SSNIT'].sum()}")

# Step 2: Parse and clean names
def parse_name(name):
    if pd.isna(name):
        return []
    
    # Convert to string and clean
    name = str(name).strip()
    
    # Remove quotes and extra spaces
    name = name.replace("'", "").replace('"', '')
    
    # Replace commas with spaces
    name = name.replace(',', ' ')
    
    # Split by whitespace and filter empty strings
    parts = [p.strip().upper() for p in name.split() if p.strip()]
    
    return parts

df['NAME_PARTS'] = df['FULL_NAME'].apply(parse_name)

# Step 3: Group by cleaned SSNIT and find matches (ONLY for valid SSNITs)
def find_name_matches(group):
    """
    Within a SSNIT group, find which records share at least one name component
    using fuzzy matching
    """
    if len(group) == 1:
        # Only one record, no need to match
        return [0]
    
    n = len(group)
    # Create clusters - records that should be grouped together
    clusters = []
    assigned = [False] * n
    
    for i in range(n):
        if assigned[i]:
            continue
        
        # Start a new cluster with record i
        cluster = [i]
        assigned[i] = True
        name_parts_i = group.iloc[i]['NAME_PARTS']
        
        # Find all records that match with record i
        for j in range(i + 1, n):
            if assigned[j]:
                continue
            
            name_parts_j = group.iloc[j]['NAME_PARTS']
            
            # Check if at least TWO name components match (fuzzy or substring)
            matches = 0
            
            for part_i in name_parts_i:
                for part_j in name_parts_j:
                    # Direct fuzzy matching with threshold of 85
                    if fuzz.ratio(part_i, part_j) >= 85:
                        matches += 1
                        break
                    # Check if one part is contained in the other (for concatenated names)
                    # e.g., "ANDYBAIDEN" contains "ANDY" or "BAIDEN"
                    elif len(part_i) >= 3 and len(part_j) >= 3:
                        if part_i in part_j or part_j in part_i:
                            matches += 1
                            break
            
            # Require at least 2 matching name parts
            if matches >= 2:
                cluster.append(j)
                assigned[j] = True
        
        clusters.append(cluster)
    
    # Assign cluster IDs
    cluster_ids = [0] * n
    for cluster_id, cluster in enumerate(clusters):
        for idx in cluster:
            cluster_ids[idx] = cluster_id
    
    return cluster_ids

print("\nGrouping by SSNIT and finding name matches (only for valid SSNITs)...")

# Initialize cluster IDs
df['CLUSTER_ID'] = -1

# Separate valid and invalid SSNIT records
valid_ssnit_df = df[~df['INVALID_SSNIT']].copy()
invalid_ssnit_df = df[df['INVALID_SSNIT']].copy()

print(f"Valid SSNIT records to process: {len(valid_ssnit_df)}")
print(f"Invalid SSNIT records (will be kept as-is): {len(invalid_ssnit_df)}")

# Group only valid SSNIT records
grouped = valid_ssnit_df.groupby('CLEANED_SSNIT')

for ssnit, group in grouped:
    if ssnit == '':  # Extra safety check
        continue
    
    indices = group.index
    cluster_ids = find_name_matches(group)
    df.loc[indices, 'CLUSTER_ID'] = cluster_ids

print(f"Clustering complete!")

# Step 4: Create final grouped dataset
# For valid SSNITs: Combine CLEANED_SSNIT and CLUSTER_ID to create unique groups
# For invalid SSNITs: Each record stays separate
valid_ssnit_df = df[~df['INVALID_SSNIT']].copy()
invalid_ssnit_df = df[df['INVALID_SSNIT']].copy()

# Process valid SSNIT records
if len(valid_ssnit_df) > 0:
    valid_ssnit_df['GROUP_KEY'] = valid_ssnit_df['CLEANED_SSNIT'] + '_' + valid_ssnit_df['CLUSTER_ID'].astype(str)
    
    result_valid = valid_ssnit_df.groupby('GROUP_KEY').agg({
        'EMPLOYEE_NUMBER': lambda x: '|'.join(sorted(set(str(v) for v in x))),
        'FULL_NAME': 'first',
        'CLEANED_SSNIT': 'first',
        'SOURCE_FILE': lambda x: '|'.join(sorted(set(str(v) for v in x)))
    }).reset_index(drop=True)
else:
    result_valid = pd.DataFrame(columns=['EMPLOYEE_NUMBER', 'FULL_NAME', 'CLEANED_SSNIT', 'SOURCE_FILE'])

# Process invalid SSNIT records - keep them as individual records
if len(invalid_ssnit_df) > 0:
    result_invalid = invalid_ssnit_df[['EMPLOYEE_NUMBER', 'FULL_NAME', 'CLEANED_SSNIT', 'SOURCE_FILE']].copy()
    result_invalid['EMPLOYEE_NUMBER'] = result_invalid['EMPLOYEE_NUMBER'].astype(str)
else:
    result_invalid = pd.DataFrame(columns=['EMPLOYEE_NUMBER', 'FULL_NAME', 'CLEANED_SSNIT', 'SOURCE_FILE'])

# Combine both results
result = pd.concat([result_valid, result_invalid], ignore_index=True)

# Rename columns
result.columns = ['EMPLOYEE_NUMBERS', 'REPRESENTATIVE_NAME', 'SSNIT_NUMBER', 'SOURCE_FILES']

# Reorder columns
result = result[['SSNIT_NUMBER', 'REPRESENTATIVE_NAME', 'EMPLOYEE_NUMBERS', 'SOURCE_FILES']]

print(f"\nFinal grouped records: {len(result)}")
print(f"  - From valid SSNITs (grouped): {len(result_valid)}")
print(f"  - From invalid SSNITs (kept as-is): {len(result_invalid)}")
print(f"Original records: {len(df)}")
print(f"Reduction: {len(df) - len(result)} records consolidated")

# Step 5: Save to Excel
output_path = r"C:\Users\spt-admin\Desktop\EMPLOYEE_NUM_PER_FIXED.xlsx"
result.to_excel(output_path, index=False, engine='openpyxl')

print(f"\nSaved to: {output_path}")

# Show sample of results
print("\nSample of grouped results:")
print(result.head(10))

# Verify the specific SSNIT case
print("\n" + "="*80)
print("VERIFICATION: Checking SSNIT 34619361")
print("="*80)
ssnit_34619361 = result[result['SSNIT_NUMBER'] == '34619361']
if len(ssnit_34619361) > 0:
    print(f"Found {len(ssnit_34619361)} distinct groups:")
    for idx, row in ssnit_34619361.iterrows():
        print(f"\nGroup {idx}:")
        print(f"  SSNIT: {row['SSNIT_NUMBER']}")
        print(f"  Name: {row['REPRESENTATIVE_NAME']}")
        print(f"  Employee IDs: {row['EMPLOYEE_NUMBERS']}")
else:
    print("SSNIT 34619361 not found in results.")